In [2]:
import hyperspy.api as hs
from matplotlib import pyplot as plt
import numpy as np
import pycrystem as pc

#Split this into grey and colour
#plt.rcParams['image.cmap'] = 'viridis'
%matplotlib tk

In [3]:
def find_max_length_peaks(peaks):
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    length_of_longest_peaks_list = 0
    for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                if peaks.data[x,y].shape[0] > length_of_longest_peaks_list:
                    length_of_longest_peaks_list = peaks.data[x,y].shape[0]
    return length_of_longest_peaks_list                

In [4]:
def generate_markers_from_peaks(peaks):
    markx,marky = [],[]
    x_size,y_size = peaks.axes_manager.navigation_shape[0],peaks.axes_manager.navigation_shape[1]
    max_peak_count = find_max_length_peaks(peaks)
    for k in np.arange(0,max_peak_count):
        mx,my = [],[]
        for x in np.arange(0,x_size):
            for y in np.arange(0,y_size):
                try:
                    mx.append(peaks.inav[x,y].data[k][1])
                    my.append(peaks.inav[x,y].data[k][0])
                except IndexError: #we have fewer than 'max_peak_count' peaks, so no marker
                    mx.append(np.nan)
                    my.append(np.nan)
        markx.append(np.asarray(mx).reshape(x_size,y_size).T)
        marky.append(np.asarray(my).reshape(x_size,y_size).T)
    
    return markx,marky    

In [5]:
nav_x = 5
nav_y = 5
sig_x = 10
sig_y = 10

In [6]:
x = np.random.rand(nav_x,nav_y,sig_x,sig_y)

In [7]:
x[0,0][7,3:6] = 5
x[0,0][5:8,5] = 5
x[1,0][7,3:7] = 5
x[0,1][7,3:7] = 5
x[1,1][6,3:6] = 5

In [8]:
dp = pc.ElectronDiffraction(x)

In [9]:
peaks = dp.find_peaks() #max,Zaf,min is dodgy

A Jupyter Widget

A Jupyter Widget

In [10]:
if sig_x == sig_y:
    peaks.data = peaks.data + (sig_x/2)
else:
    raise NotImplementedError('Needs a square signal')

In [11]:
markx,marky = generate_markers_from_peaks(peaks)

In [20]:
import itertools
markxT = [None] * find_max_length_peaks(peaks)
markyT = [None] * find_max_length_peaks(peaks)

a = np.array(list(itertools.zip_longest(*np.concatenate(peaks.data),fillvalue=[np.nan,np.nan])))
a = a.reshape(len(markxT),nav_x,nav_y,2) #getting the 2D nav back
xy = np.transpose(a,[3,0,1,2]) #switching for selection
x = xy[1]
y = xy[0]
for cnt in [0,1,2,3]:
    markxT[cnt] = x[cnt]
    markyT[cnt] = y[cnt]

In [22]:
dp.plot() #the requirement to include this is a bug.
for mx,my in zip(markx,marky):
    m = hs.markers.point(x=mx,y=my,color='r')
    dp.add_marker(m,plot_marker=True,permanent=False)
for mx,my in zip(markxT,markyT):
    m = hs.markers.point(x=mx,y=my,color='gold',marker='o')
    dp.add_marker(m,plot_marker=True,permanent=False)

ValueError: None of the coordinates have value